In [1]:
def str2mem(line):
    return [int(i) for i in line.split(",")]

In [2]:
def get_opcode_and_modes(number):
    s = "{:05d}".format(number)
    opcode = int(s[-2:])
    modes = list(map(int, s[:3]))
    modes.reverse()
    if modes[2] == 1:
        raise ValueError("Mode of parameter 3 cannot not be 1")
    return opcode, modes

In [3]:
def get_params(memory, ptr, modes, ptypes):
    values = []
    params = memory[ptr + 1: ptr + len(modes) + 1]
    for param, mode, ptype in zip(params, modes, ptypes):
        if mode == 1 or ptype == 'w':
            values.append(param)
        elif mode == 0:
            values.append(memory[param])
    return values

In [4]:
def intcode(memory, input_value=None):
    output = ''
    ptr = 0
    while True:
        opcode, modes = get_opcode_and_modes(memory[ptr])
        if opcode == 1:
            p1, p2, p3 = get_params(memory, ptr, modes, 'rrw')
            memory[p3] = p1 + p2
            ptr += 4
        elif opcode == 2:
            p1, p2, p3 = get_params(memory, ptr, modes, 'rrw')
            memory[p3] = p1 * p2
            ptr += 4
        elif opcode == 3:
            p1, *_ = get_params(memory, ptr, modes, 'w')
            memory[p1] = input_value
            ptr += 2
        elif opcode == 4:
            output, *_ = get_params(memory, ptr, modes, 'r')
            print("Output:", output)
            ptr += 2
        elif opcode == 5:
            p1, p2 = get_params(memory, ptr, modes, 'rr')
            if p1:
                ptr = p2
            else:
                ptr += 3
        elif opcode == 6:
            p1, p2 = get_params(memory, ptr, modes, 'rr')
            if not p1:
                ptr = p2
            else:
                ptr += 3
        elif opcode == 7:
            p1, p2, p3 = get_params(memory, ptr, modes, 'rrw')
            memory[p3] = int(p1 < p2)
            ptr += 4
        elif opcode == 8:
            p1, p2, p3 = get_params(memory, ptr, modes, 'rrw')
            memory[p3] = int(p1 == p2)
            ptr += 4
        elif opcode == 99:
            break
        else:
            raise ValueError("Unknown opcode: {}".format(opcode))
            
    return memory, output

In [5]:
# Tests
assert intcode(str2mem("1,9,10,3,2,3,11,0,99,30,40,50"))[0] == str2mem("3500,9,10,70,2,3,11,0,99,30,40,50")
assert intcode(str2mem("1,0,0,0,99"))[0] == str2mem("2,0,0,0,99")
assert intcode(str2mem("1,1,1,4,99,5,6,0,99"))[0] == str2mem("30,1,1,4,2,5,6,0,99")

assert intcode(str2mem("1002,4,3,4,33"))[0] == str2mem("1002,4,3,4,99")

In [6]:
# Is input equal to 8?
assert intcode(str2mem("3,9,8,9,10,9,4,9,99,-1,8"), input_value=8)[1] == 1
assert intcode(str2mem("3,9,8,9,10,9,4,9,99,-1,8"), input_value=7)[1] == 0
assert intcode(str2mem("3,9,8,9,10,9,4,9,99,-1,8"), input_value=9)[1] == 0

Output: 1
Output: 0
Output: 0


In [7]:
# Output 0 if input is 0, otherwise output 1
assert intcode(str2mem("3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9"), input_value=0)[1] == 0
assert intcode(str2mem("3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9"), input_value=-1)[1] == 1
assert intcode(str2mem("3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9"), input_value=1)[1] == 1

assert intcode(str2mem("3,3,1105,-1,9,1101,0,0,12,4,12,99,1"), input_value=0)[1] == 0
assert intcode(str2mem("3,3,1105,-1,9,1101,0,0,12,4,12,99,1"), input_value=-1)[1] == 1
assert intcode(str2mem("3,3,1105,-1,9,1101,0,0,12,4,12,99,1"), input_value=1)[1] == 1

Output: 0
Output: 1
Output: 1
Output: 0
Output: 1
Output: 1


# Part 1

In [8]:
with open("day05.input") as file:
    memory = str2mem(file.readline().strip())

memory, output = intcode(memory, input_value=1)
output

Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 5577461


5577461

# Part 2

In [9]:
prog = "3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4"\
       ",20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99"

assert intcode(str2mem(prog), input_value=7)[1] == 999
assert intcode(str2mem(prog), input_value=8)[1] == 1000
assert intcode(str2mem(prog), input_value=9)[1] == 1001

Output: 999
Output: 1000
Output: 1001


In [10]:
with open("day05.input") as file:
    memory = str2mem(file.readline().strip())

memory, output = intcode(memory, input_value=5)
output

Output: 7161591


7161591